In [108]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy import stats
import statsmodels.api as sm


In [109]:
#Scales dataset and metric to evaluate

scales = pd.read_excel("filled_scales_BEBRASK_RETOS.xlsx")

In [110]:
trend_data = pd.read_excel("trend_dataset.xlsx")

trend_data["Together_Match"] = trend_data["AVG_Match"] + trend_data["Trend_Match"]*27
trend_data["Together_No_Match"] = trend_data["AVG_No_Match"] + trend_data["Trend_No_Match"]*18

baseline_data = pd.read_excel("RETOS_BEBRASK_Baseline.xlsx")

In [111]:
features_dataset = pd.merge(left = baseline_data,right=trend_data, left_on="Subject",right_on="Subject ID").drop("Subject ID",axis=1)

In [112]:
scaler = StandardScaler()
scaler.fit(features_dataset.drop("Subject",axis=1))
features_dataset_scaled = scaler.transform(features_dataset.drop("Subject",axis=1))

In [113]:
features_dataset_scaled = pd.DataFrame(features_dataset_scaled)
features_dataset_scaled.insert(0, 'Subject', features_dataset['Subject'])

In [114]:
features_dataset_scaled.columns = features_dataset.columns

In [115]:
features_dataset_scaled

,Subject,Mean_Rating0,Dif_Match,Cor_Pred_Like,Trend_Match,Trend_No_Match,AVG_Match,AVG_No_Match,Together_Match,Together_No_Match
0,PREDWELL_RETOS-1-1,1.993171,0.632545,-0.926969,-0.658026,-1.132586,1.656228,1.562985,0.832344,-0.232422
1,PREDWELL_RETOS-10-1,0.502669,-0.852105,-0.534772,-1.059467,0.813927,0.833654,-0.149700,-0.734860,1.305163
2,PREDWELL_RETOS-1001-1,0.968451,0.274406,-0.064176,0.687980,-0.762995,0.085340,0.915671,1.186616,-0.317060
3,PREDWELL_RETOS-1002-1,1.247920,-0.253036,0.379229,1.113034,-1.280423,-0.223126,1.684357,1.510865,-0.359379
4,PREDWELL_RETOS-1003-1,-0.894676,0.066034,-1.081741,-0.539956,0.641452,0.011080,-1.039758,-0.842943,-0.049041
...,...,...,...,...,...,...,...,...,...,...
144,PREDWELL_RETOS-5-1,-0.149425,-0.305129,-0.493753,0.876893,-1.452898,-0.794358,1.252814,0.490081,-1.177544
145,PREDWELL_RETOS-6-1,-0.615207,-0.747919,0.568578,1.845073,-0.294846,-1.794014,0.187442,0.892390,-0.317060
146,PREDWELL_RETOS-7-1,0.130044,-0.148850,0.286641,-0.421885,-0.516601,0.370956,0.484128,-0.248486,-0.373485
147,PREDWELL_RETOS-8-1,0.595826,-0.852105,-0.645545,-0.091287,-0.393404,0.131038,0.956128,0.003708,0.402361


In [116]:
scales.drop(["SUBJECT_CODE","Age"],axis=1,inplace=True)
metrics_columns = scales.drop("EPRIME_CODE",axis=1).columns.values

In [117]:
feature_scales = pd.merge(left = features_dataset_scaled,right=scales, left_on="Subject",right_on="EPRIME_CODE").drop(["EPRIME_CODE","Subject"],axis=1)

In [118]:
features_columns = features_dataset.drop("Subject",axis=1).columns
dic_features_scales = {}
for metric in metrics_columns:
    target = feature_scales[metric]
    dic_features_scales[metric] = {}
    for feature in features_columns:
        
        X = feature_scales[feature]
        X2 = sm.add_constant(X)
        est = sm.OLS(target, X2)
        est2 = est.fit()
        p_values = est2.summary2().tables[1]['P>|t|']
        dic_features_scales[metric].update({feature:p_values.iloc[1]})

In [119]:
df_features_scales = pd.DataFrame(dic_features_scales)
df_features_scales.mean(axis=1)


Mean_Rating0         0.467371
Dif_Match            0.455141
Cor_Pred_Like        0.606172
Trend_Match          0.441686
Trend_No_Match       0.634640
AVG_Match            0.540235
AVG_No_Match         0.535258
Together_Match       0.453699
Together_No_Match    0.616108
dtype: float64

In [139]:
# Assuming feature_scales is your DataFrame with features as columns and metrics as rows
dic_mean_comparison_85 = {}

for metric in metrics_columns:
    dic_mean_comparison_85[metric] = {}

    for feature in features_columns:
        
        high_values = feature_scales[feature_scales[feature] > 1.04][metric] #Per sobre 85%
        low_values = feature_scales[feature_scales[feature] <= 1.04][metric] #Per sota 85%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison_85[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': p_value}


# Convert results to a DataFrame for easier visualization and analysis
df_mean_comparison_85 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison_85[i][j] 
                                             for i in dic_mean_comparison_85.keys() 
                                             for j in dic_mean_comparison_85[i].keys()},
                                            orient='index')
print("85%")
df_mean_comparison_85[df_mean_comparison_85["p_value"]<0.10]

85%


high_mean  size_high   low_mean  size_low  \
PA        Trend_No_Match     35.631579         19  32.892308       130   
          Together_No_Match  35.190476         21  32.921875       128   
NA.       Dif_Match          18.714286         21  20.515625       128   
          AVG_Match          22.666667         21  19.867188       128   
ERQ_CR    Dif_Match           5.412222         21   4.807370       128   
          AVG_Match           5.214603         21   4.839792       128   
UPPSP_NU  AVG_No_Match       10.120000         25   8.637097       124   
UPPSP_PU  AVG_No_Match       10.560000         25   9.709677       124   
UPPSP_SS  Dif_Match          12.666667         21  10.554688       128   
UPPSP_PMD Cor_Pred_Like       6.758621         29   7.561712       120   
          Trend_No_Match      6.705548         19   7.507692       130   
          AVG_Match           6.523810         21   7.550042       128   
BIS       Trend_No_Match     20.684211         19  21.761538       130   
          Together_No_Match  20.142857         21  21.867188       128   
BAS_RR    Mean_Rating0       18.181818         22  17.346457       127   
BAS_D     Trend_No_Match     12.842105         19  11.730769       130   
TEPS_CF   Dif_Match          38.714286         21  35.117188       128   
FS        Dif_Match          48.952381         21  47.179688       128   
LOT_R     Dif_Match          16.809524         21  14.593750       128   
RRQ_Rum   Together_No_Match   3.024286         21   3.370130       128   
RRQ_Ref   AVG_No_Match        3.670400         25   3.305538       124   
          Together_No_Match   3.774444         21   3.299870       128   
ASI_P     Dif_Match           3.380952         21   4.935283       128   
ASI_C     Dif_Match           1.761905         21   3.840213       128   
SPQ       Mean_Rating0        1.681818         22   2.307087       127   
MSSB_DES  Dif_Match           1.476190         21   2.382812       128   

                              p_value  
PA        Trend_No_Match     0.061146  
          Together_No_Match  0.068908  
NA.       Dif_Match          0.073243  
          AVG_Match          0.012831  
ERQ_CR    Dif_Match          0.000245  
          AVG_Match          0.056315  
UPPSP_NU  AVG_No_Match       0.016544  
UPPSP_PU  AVG_No_Match       0.061120  
UPPSP_SS  Dif_Match          0.000973  
UPPSP_PMD Cor_Pred_Like      0.063913  
          Trend_No_Match     0.045547  
          AVG_Match          0.042609  
BIS       Trend_No_Match     0.099454  
          Together_No_Match  0.005758  
BAS_RR    Mean_Rating0       0.048977  
BAS_D     Trend_No_Match     0.022606  
TEPS_CF   Dif_Match          0.054366  
FS        Dif_Match          0.072504  
LOT_R     Dif_Match          0.003343  
RRQ_Rum   Together_No_Match  0.096162  
RRQ_Ref   AVG_No_Match       0.075809  
          Together_No_Match  0.017997  
ASI_P     Dif_Match          0.080846  
ASI_C     Dif_Match          0.000625  
SPQ       Mean_Rating0       0.040130  
MSSB_DES  Dif_Match          0.048599

In [140]:
df_mean_comparison_85.groupby(level=1)['p_value'].mean()


AVG_Match            0.440441
AVG_No_Match         0.516895
Cor_Pred_Like        0.546297
Dif_Match            0.351162
Mean_Rating0         0.537520
Together_Match       0.572697
Together_No_Match    0.501403
Trend_Match          0.585963
Trend_No_Match       0.494199
Name: p_value, dtype: float64

In [141]:


# Assuming feature_scales is your DataFrame with features as columns and metrics as rows
dic_mean_comparison_90 = {}

for metric in metrics_columns:
    dic_mean_comparison_90[metric] = {}

    for feature in features_columns:
        
        high_values = feature_scales[feature_scales[feature] > 1.28][metric] #Per sobre 90%
        low_values = feature_scales[feature_scales[feature] <= 1.28][metric] #Per sota 90%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison_90[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': p_value}


# Convert results to a DataFrame for easier visualization and analysis
df_mean_comparison_90 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison_90[i][j] 
                                             for i in dic_mean_comparison_90.keys() 
                                             for j in dic_mean_comparison_90[i].keys()},
                                            orient='index')
print("90%")
df_mean_comparison_90[df_mean_comparison_90["p_value"]<0.1]

90%


high_mean  size_high   low_mean  size_low  \
PA        Trend_No_Match     36.090909         11  33.014493       138   
          Together_Match     36.384615         13  32.941176       136   
NA.       AVG_Match          23.066667         15  19.947761       134   
ERQ_CR    Dif_Match           5.416429         14   4.838296       135   
          AVG_Match           5.445333         15   4.830746       134   
ERQ_ES    Together_Match      2.134615         13   3.119485       136   
UPPSP_NU  Mean_Rating0       10.700000         10   8.755396       139   
UPPSP_PU  Mean_Rating0       10.800000         10   9.784173       139   
UPPSP_SS  Dif_Match          13.071429         14  10.622222       135   
UPPSP_PMD Cor_Pred_Like       6.700000         20   7.514771       129   
          AVG_Match           6.400000         15   7.517951       134   
UPPSP_PSV Dif_Match           5.642857         14   7.192593       135   
BIS       Cor_Pred_Like      23.550000         20  21.325581       129   
          AVG_Match          22.866667         15  21.485075       134   
          Together_No_Match  20.111111         18  21.832061       131   
BAS_RR    Together_Match     18.461538         13  17.375000       136   
BAS_D     Trend_Match        12.933333         15  11.753731       134   
LOT_R     Dif_Match          17.428571         14  14.644444       135   
RRQ_Rum   AVG_Match           3.800000         15   3.267811       134   
          Together_Match      2.782821         13   3.372868       136   
RRQ_Ref   Dif_Match           2.851190         14   3.420222       135   
          Trend_No_Match      3.743333         11   3.336739       138   
          Together_No_Match   3.829074         18   3.303232       131   
ASI_P     Dif_Match           2.357143         14   4.960861       135   
ASI_C     Dif_Match           1.857143         14   3.722573       135   
ASI_S     AVG_Match          11.000000         15   7.977612       134   
SPQ       AVG_Match           3.466667         15   2.074627       134   
          Together_Match      1.538462         13   2.279412       136   
SPQ_IR    AVG_Match           3.600000         15   2.544776       134   
MSSB_POS  Trend_No_Match      1.090909         11   1.992754       138   
MSSB_NEG  Dif_Match           1.214286         14   1.918519       135   
          Trend_No_Match      1.000000         11   1.920290       138   
MSSB_DES  Dif_Match           1.000000         14   2.385185       135   

                              p_value  
PA        Trend_No_Match     0.064588  
          Together_Match     0.093602  
NA.       AVG_Match          0.026458  
ERQ_CR    Dif_Match          0.003555  
          AVG_Match          0.007456  
ERQ_ES    Together_Match     0.002806  
UPPSP_NU  Mean_Rating0       0.044600  
UPPSP_PU  Mean_Rating0       0.009083  
UPPSP_SS  Dif_Match          0.001381  
UPPSP_PMD Cor_Pred_Like      0.078667  
          AVG_Match          0.071906  
UPPSP_PSV Dif_Match          0.028825  
BIS       Cor_Pred_Like      0.007958  
          AVG_Match          0.034121  
          Together_No_Match  0.012096  
BAS_RR    Together_Match     0.027854  
BAS_D     Trend_Match        0.087353  
LOT_R     Dif_Match          0.001347  
RRQ_Rum   AVG_Match          0.006796  
          Together_Match     0.045025  
RRQ_Ref   Dif_Match          0.052374  
          Trend_No_Match     0.087529  
          Together_No_Match  0.009227  
ASI_P     Dif_Match          0.012790  
ASI_C     Dif_Match          0.009133  
ASI_S     AVG_Match          0.092136  
SPQ       AVG_Match          0.005225  
          Together_Match     0.035455  
SPQ_IR    AVG_Match          0.056680  
MSSB_POS  Trend_No_Match     0.021495  
MSSB_NEG  Dif_Match          0.066608  
          Trend_No_Match     0.009825  
MSSB_DES  Dif_Match          0.004512

In [142]:
df_mean_comparison_90.groupby(level=1)['p_value'].mean()

AVG_Match            0.395466
AVG_No_Match         0.589679
Cor_Pred_Like        0.595405
Dif_Match            0.328443
Mean_Rating0         0.568058
Together_Match       0.482768
Together_No_Match    0.555243
Trend_Match          0.614176
Trend_No_Match       0.444896
Name: p_value, dtype: float64

In [143]:
dic_mean_comparison = {}

for metric in metrics_columns:
    dic_mean_comparison[metric] = {}

    for feature in features_columns:
        
        high_values = feature_scales[feature_scales[feature] > 1.6][metric] #Per sobre 95%
        low_values = feature_scales[feature_scales[feature] <= 1.6][metric] #Per sota 95%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': p_value}


# Convert results to a DataFrame for easier visualization and analysis
df_mean_comparison_95 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison[i][j] 
                                             for i in dic_mean_comparison.keys() 
                                             for j in dic_mean_comparison[i].keys()},
                                            orient='index')
print("95%")
df_mean_comparison_95[df_mean_comparison_95["p_value"]<0.1]



95%


high_mean  size_high   low_mean  size_low  \
PA        Together_Match     39.125000          8  32.907801       141   
NA.       Together_Match     16.500000          8  20.475177       141   
ERQ_CR    Dif_Match           5.642381          7   4.855657       142   
          AVG_Match           5.480417          8   4.859267       141   
ERQ_ES    AVG_No_Match        2.527778          9   3.066071       140   
          Together_Match      1.750000          8   3.106383       141   
UPPSP_NU  Mean_Rating0       10.777778          9   8.764286       140   
UPPSP_PU  Mean_Rating0       10.666667          9   9.800000       140   
          AVG_Match          10.750000          8   9.801418       141   
          Together_No_Match   8.777778          9   9.921429       140   
UPPSP_PMD Cor_Pred_Like       6.363636         11   7.488445       138   
          Together_No_Match   6.333333          9   7.474324       140   
BIS       Cor_Pred_Like      24.909091         11  21.362319       138   
BAS_RR    Trend_Match        18.777778          9  17.385714       140   
          Together_No_Match  18.555556          9  17.400000       140   
BAS_D     Trend_Match        13.444444          9  11.771429       140   
TEPS_CF   Trend_No_Match     40.500000          6  35.419580       143   
FS        Dif_Match          50.142857          7  47.295775       142   
          Together_Match     50.625000          8  47.248227       141   
LOT_R     Dif_Match          17.428571          7  14.781690       142   
          Together_Match     17.625000          8  14.751773       141   
RRQ_Rum   Cor_Pred_Like       3.902121         11   3.275097       138   
          AVG_Match           3.823750          8   3.292884       141   
          Together_Match      2.427917          8   3.372080       141   
RRQ_Ref   Dif_Match           2.392857          7   3.414765       142   
          Trend_No_Match      4.056667          6   3.337809       143   
          Together_No_Match   3.908519          9   3.331929       140   
ASI_P     Dif_Match           1.571429          7   4.871241       142   
          Together_Match      1.875000          8   4.877420       141   
ASI_C     Dif_Match           1.000000          7   3.672868       142   
          Together_Match      1.000000          8   3.691825       141   
ASI_S     Cor_Pred_Like      11.272727         11   8.043478       138   
          AVG_No_Match        5.000000          9   8.492857       140   
          Together_Match      3.625000          8   8.546099       141   
SPQ       Cor_Pred_Like       3.363636         11   2.123188       138   
          AVG_Match           3.000000          8   2.170213       141   
          Together_Match      1.375000          8   2.262411       141   
SPQ_IR    AVG_Match           3.500000          8   2.602837       141   
MSSB_NEG  Dif_Match           1.000000          7   1.894366       142   
MSSB_DES  Dif_Match           1.000000          7   2.316901       142   

                              p_value  
PA        Together_Match     0.021389  
NA.       Together_Match     0.003203  
ERQ_CR    Dif_Match          0.011265  
          AVG_Match          0.050537  
ERQ_ES    AVG_No_Match       0.099975  
          Together_Match     0.000040  
UPPSP_NU  Mean_Rating0       0.060888  
UPPSP_PU  Mean_Rating0       0.023263  
          AVG_Match          0.083300  
          Together_No_Match  0.015626  
UPPSP_PMD Cor_Pred_Like      0.074001  
          Together_No_Match  0.090766  
BIS       Cor_Pred_Like      0.002100  
BAS_RR    Trend_Match        0.042902  
          Together_No_Match  0.086141  
BAS_D     Trend_Match        0.062894  
TEPS_CF   Trend_No_Match     0.001885  
FS        Dif_Match          0.039681  
          Together_Match     0.048723  
LOT_R     Dif_Match          0.066358  
          Together_Match     0.031861  
RRQ_Rum   Cor_Pred_Like      0.016472  
          AVG_Match          0.085894  
          Together_Match     0.000

In [144]:
df_mean_comparison_95.groupby(level=1)['p_value'].mean()


AVG_Match            0.421083
AVG_No_Match         0.523863
Cor_Pred_Like        0.435817
Dif_Match            0.415787
Mean_Rating0         0.533614
Together_Match       0.319518
Together_No_Match    0.507024
Trend_Match          0.620207
Trend_No_Match       0.538885
Name: p_value, dtype: float64

In [145]:
dic_mean_comparison_5 = {}

for metric in metrics_columns:
    dic_mean_comparison_5[metric] = {}

    for feature in features_columns:
        
        high_values = feature_scales[feature_scales[feature] < -1.6][metric] #Per sota 5%
        low_values = feature_scales[feature_scales[feature] >= -1.6][metric] #Per sobre 5%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison_5[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': p_value}


# Convert results to a DataFrame for easier visualization and analysis
print("5%")
df_mean_comparison_5 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison_5[i][j] 
                                             for i in dic_mean_comparison_5.keys() 
                                             for j in dic_mean_comparison_5[i].keys()},
                                            orient='index')

df_mean_comparison_5[df_mean_comparison_5["p_value"]<0.1]

/home/diego/PycharmProjects/CasLAB-Predictive-Processing/venv/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


5%


high_mean  size_high   low_mean  size_low  \
PA        Mean_Rating0       28.200000          5  33.416667       144   
NA.       Trend_No_Match     17.600000         10  20.453237       139   
ERQ_ES    Mean_Rating0        1.900000          5   3.072917       144   
UPPSP_SS  Together_Match      9.181818         11  10.985507       138   
UPPSP_PSV AVG_No_Match       11.500000          2   6.986395       147   
BIS       Mean_Rating0       25.000000          5  21.506944       144   
          Dif_Match          19.142857          7  21.746479       142   
BAS_RR    AVG_Match          19.166667          6  17.398601       143   
          AVG_No_Match       20.000000          2  17.435374       147   
BAS_D     AVG_No_Match       13.000000          2  11.857143       147   
BAS_FS    AVG_Match          14.833333          6  12.349650       143   
          Together_Match     11.363636         11  12.536232       138   
TEPS_AF   Cor_Pred_Like      52.250000          4  43.268966       145   
          Together_No_Match  47.666667          9  43.242857       140   
TEPS_CF   AVG_Match          40.000000          6  35.440559       143   
          Together_No_Match  38.444444          9  35.442857       140   
SHS       Mean_Rating0        3.800000          5   4.296875       144   
          Trend_No_Match      4.600000         10   4.257194       139   
LOT_R     Dif_Match          13.428571          7  14.978873       142   
RRQ_Rum   Trend_No_Match      2.758333         10   3.361894       139   
RRQ_Ref   Trend_No_Match      4.008333         10   3.320600       139   
ASI_P     Cor_Pred_Like       1.250000          4   4.811836       145   
          Trend_No_Match      2.100000         10   4.904433       139   
ASI_C     Together_Match      7.636364         11   3.221357       138   
ASI_S     Trend_No_Match      4.800000         10   8.532374       139   
MSSB_NEG  Together_Match      1.272727         11   1.898551       138   
          Together_No_Match   1.333333          9   1.885714       140   

                                  p_value  
PA        Mean_Rating0       4.771396e-02  
NA.       Trend_No_Match     8.950952e-02  
ERQ_ES    Mean_Rating0       3.894219e-03  
UPPSP_SS  Together_Match     7.622709e-02  
UPPSP_PSV AVG_No_Match       3.701629e-02  
BIS       Mean_Rating0       2.677734e-02  
          Dif_Match          1.370842e-02  
BAS_RR    AVG_Match          4.181800e-02  
          AVG_No_Match       8.388936e-35  
BAS_D     AVG_No_Match       8.810011e-09  
BAS_FS    AVG_Match          1.598802e-09  
          Together_Match     7.538289e-02  
TEPS_AF   Cor_Pred_Like      6.717008e-03  
          Together_No_Match  6.349384e-02  
TEPS_CF   AVG_Match          2.248646e-02  
          Together_No_Match  7.863043e-02  
SHS       Mean_Rating0       3.656072e-02  
          Trend_No_Match     7.254335e-02  
LOT_R     Dif_Match          6.109776e-02  
RRQ_Rum   Trend_No_Match     6.078809e-02  
RRQ_Ref   Trend_No_Match     4.980010e-02  
ASI_P     Cor_Pred_Like      5.270034e-04  
          Trend_No_Match     3.716884e-03  
ASI_C     Together_Match     9.810251e-02  
ASI_S     Trend_No_Match     6.473136e-03  
MSSB_NEG  Together_Match     9.293845e-02  
          Together_No_Match  2.960341e-02

In [146]:
df_mean_comparison_5.groupby(level=1)['p_value'].mean()


AVG_Match            0.544163
AVG_No_Match         0.507565
Cor_Pred_Like        0.561616
Dif_Match            0.529052
Mean_Rating0         0.512888
Together_Match       0.434878
Together_No_Match    0.386037
Trend_Match          0.569449
Trend_No_Match       0.438827
Name: p_value, dtype: float64

In [149]:
dic_mean_comparison_10 = {}

for metric in metrics_columns:
    dic_mean_comparison_10[metric] = {}

    for feature in features_columns:
        
        high_values = feature_scales[feature_scales[feature] < -1.28][metric] #Per sota 10%
        low_values = feature_scales[feature_scales[feature] >= -1.28][metric] #Per sobre 10%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison_10[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': p_value}


# Convert results to a DataFrame for easier visualization and analysis
df_mean_comparison_10 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison_10[i][j] 
                                             for i in dic_mean_comparison_10.keys() 
                                             for j in dic_mean_comparison_10[i].keys()},
                                            orient='index')
print("10%")

df_mean_comparison_10[df_mean_comparison_10["p_value"]<0.1]

10%


high_mean  size_high   low_mean  size_low  \
PA       Mean_Rating0       29.533333         15  33.656716       134   
NA.      Dif_Match          22.000000         12  20.109489       137   
ERQ_CR   Dif_Match           4.444722         12   4.931849       137   
ERQ_ES   Mean_Rating0        2.566667         15   3.085821       134   
         Together_Match      2.464286         14   3.092593       135   
         Together_No_Match   2.539474         19   3.105769       130   
UPPSP_PU AVG_Match          11.050000         20   9.666667       129   
UPPSP_SS AVG_No_Match        8.888889          9  10.978571       140   
         Together_Match      9.285714         14  11.014815       135   
BIS      Mean_Rating0       23.000000         15  21.470149       134   
         Dif_Match          19.833333         12  21.781022       137   
BAS_RR   AVG_No_Match       18.444444          9  17.407143       140   
BAS_FS   Together_Match     11.500000         14  12.548148       135   
TEPS_AF  AVG_No_Match       47.444444          9  43.257143       140   
         Together_No_Match  46.368421         19  43.092308       130   
RRQ_Ref  Trend_No_Match      3.811556         15   3.316965       134   
SPQ      AVG_Match           1.750000         20   2.286822       129   
SPQ_IR   AVG_No_Match        3.555556          9   2.592857       140   
MSSB_POS Together_Match      3.071429         14   1.807407       135   
MSSB_NEG Together_Match      1.214286         14   1.918519       135   
         Together_No_Match   1.368421         19   1.923077       130   

                             p_value  
PA       Mean_Rating0       0.042053  
NA.      Dif_Match          0.096622  
ERQ_CR   Dif_Match          0.040691  
ERQ_ES   Mean_Rating0       0.056328  
         Together_Match     0.047859  
         Together_No_Match  0.071601  
UPPSP_PU AVG_Match          0.041523  
UPPSP_SS AVG_No_Match       0.087425  
         Together_Match     0.036179  
BIS      Mean_Rating0       0.073733  
         Dif_Match          0.053588  
BAS_RR   AVG_No_Match       0.052509  
BAS_FS   Together_Match     0.065012  
TEPS_AF  AVG_No_Match       0.049053  
         Together_No_Match  0.085388  
RRQ_Ref  Trend_No_Match     0.066845  
SPQ      AVG_Match          0.094181  
SPQ_IR   AVG_No_Match       0.051475  
MSSB_POS Together_Match     0.088423  
MSSB_NEG Together_Match     0.034858  
         Together_No_Match  0.051484

In [150]:
df_mean_comparison_10.groupby(level=1)['p_value'].mean()


AVG_Match            0.570490
AVG_No_Match         0.418113
Cor_Pred_Like        0.489073
Dif_Match            0.602982
Mean_Rating0         0.385582
Together_Match       0.472063
Together_No_Match    0.494912
Trend_Match          0.532096
Trend_No_Match       0.514168
Name: p_value, dtype: float64

In [151]:
# Assuming feature_scales is your DataFrame with features as columns and metrics as rows
dic_mean_comparison_15 = {}

for metric in metrics_columns:
    dic_mean_comparison_15[metric] = {}

    for feature in features_columns:
        
        high_values = feature_scales[feature_scales[feature] < -1.04][metric] #Per sota 15%
        low_values = feature_scales[feature_scales[feature] >= -1.04][metric] #Per sobre 15%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison_15[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': p_value}


# Convert results to a DataFrame for easier visualization and analysis
df_mean_comparison_15 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison_15[i][j] 
                                             for i in dic_mean_comparison_15.keys() 
                                             for j in dic_mean_comparison_15[i].keys()},
                                            orient='index')
print("85%")
df_mean_comparison_15[df_mean_comparison_15["p_value"]<0.10]

85%


high_mean  size_high   low_mean  size_low  \
PA        Mean_Rating0       29.900000         20  33.759690       129   
          Dif_Match          35.823529         17  32.909091       132   
          AVG_Match          30.954545         22  33.637795       127   
ERQ_CR    Mean_Rating0        4.542000         20   4.946977       129   
          Together_No_Match   5.284444         24   4.817387       125   
ERQ_ES    Mean_Rating0        2.487500         20   3.118217       129   
          Together_Match      2.663043         23   3.101190       126   
UPPSP_PU  AVG_Match          11.000000         22   9.653543       127   
          Together_Match      9.130435         23   9.984127       126   
UPPSP_SS  Trend_Match         9.920000         25  11.040323       124   
          Together_Match      9.652174         23  11.071429       126   
UPPSP_PMD Trend_Match         6.600000         25   7.567786       124   
UPPSP_PSV AVG_Match           8.227273         22   6.842520       127   
BIS       Mean_Rating0       23.300000         20  21.364341       129   
BAS_RR    AVG_No_Match       18.437500         16  17.353383       133   
TEPS_AF   Trend_No_Match     46.041667         24  43.024000       125   
          AVG_No_Match       46.937500         16  43.097744       133   
LOT_R     Trend_No_Match     16.166667         24  14.664000       125   
ASI_C     Mean_Rating0        6.200000         20   3.136026       129   
ASI_S     Mean_Rating0       10.750000         20   7.899225       129   
SPQ       Trend_Match         2.800000         25   2.096774       124   
MSSB_DES  AVG_Match           3.590909         22   2.023622       127   

                              p_value  
PA        Mean_Rating0       0.024928  
          Dif_Match          0.076193  
          AVG_Match          0.069265  
ERQ_CR    Mean_Rating0       0.095583  
          Together_No_Match  0.030373  
ERQ_ES    Mean_Rating0       0.010794  
          Together_Match     0.088016  
UPPSP_PU  AVG_Match          0.032401  
          Together_Match     0.066813  
UPPSP_SS  Trend_Match        0.077870  
          Together_Match     0.026432  
UPPSP_PMD Trend_Match        0.032076  
UPPSP_PSV AVG_Match          0.059707  
BIS       Mean_Rating0       0.011348  
BAS_RR    AVG_No_Match       0.007295  
TEPS_AF   Trend_No_Match     0.095248  
          AVG_No_Match       0.025112  
LOT_R     Trend_No_Match     0.021199  
ASI_C     Mean_Rating0       0.057300  
ASI_S     Mean_Rating0       0.089660  
SPQ       Trend_Match        0.074555  
MSSB_DES  AVG_Match          0.051064

In [152]:
df_mean_comparison_15.groupby(level=1)['p_value'].mean()

AVG_Match            0.494690
AVG_No_Match         0.440877
Cor_Pred_Like        0.524054
Dif_Match            0.554948
Mean_Rating0         0.340630
Together_Match       0.410309
Together_No_Match    0.583480
Trend_Match          0.416062
Trend_No_Match       0.601684
Name: p_value, dtype: float64